### Cleanup Repositories

Call `Benchmark.cleanRepository` Operation

In [1]:
!curl -H 'Content-Type:application/json+nxrequest' -X POST -d '{"params":{"repositoryName":"*"},"context":{}}' -u $NXUSER:$NXPWD http://127.0.0.1:8080/nuxeo/api/v1/automation/Benchmark.cleanRepository


{"entity-type":"string","value":"85f913a8-dc26-41ab-b0e9-2523e8d4dfae"}

For safety reason, this call should return a token that you need to provide in a second call so that the delete is executed.

In [2]:
!curl -H 'Content-Type:application/json+nxrequest' -X POST -d '{"params":{"repositoryName":"*", "confirmationKey":"85f913a8-dc26-41ab-b0e9-2523e8d4dfae"},"context":{}}' -u $NXUSER:$NXPWD http://127.0.0.1:8080/nuxeo/api/v1/automation/Benchmark.cleanRepository


{"entity-type":"string","value":"Cleaned up MongoDB and ES for default us-east us-west archives."}

Then you may want to also clear the Audit Index by calling directly ES API


In [6]:
!curl -X DELETE $ES_SERVER/nuxeo-audit -H 'Content-Type: application/json' 


{"acknowledged":true}

### MongoDB indexes

 - some index need to be recreated in sparse mode
 - some index need to be added


In [7]:
js=""
for col in ["us-east", "us-west", "archives"]:
    js = js + """

print("************************");
print("update collection $COL");

// Switch default indexes in Sparse mode
db["$COL"].dropIndex( { "ecm:versionSeriesId": 1 });
db["$COL"].createIndex( { "ecm:versionSeriesId": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "ecm:proxyTargetId": 1 });
db["$COL"].createIndex( { "ecm:proxyTargetId": 1 }, { sparse: true }, { background: true} );
db["$COL"].dropIndex( { "ecm:proxyVersionSeriesId": 1 });
db["$COL"].createIndex( { "ecm:proxyVersionSeriesId": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "ecm:racl": 1 });
db["$COL"].createIndex( { "ecm:racl": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "ecm:retainUntil": 1 });
db["$COL"].createIndex( { "ecm:retainUntil": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "ecm:fulltextJobId": 1 });
db["$COL"].createIndex( { "ecm:fulltextJobId": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "ecm:acp.acl.user": 1 });
db["$COL"].createIndex( { "ecm:acp.acl.user": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "ecm:acp.acl.status": 1 });
db["$COL"].createIndex( { "ecm:acp.acl.status": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "dc:modified_": -1 });
db["$COL"].createIndex( { "dc:modified": -1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "rend:renditionName": 1 });
db["$COL"].createIndex( { "rend:renditionName": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "rend:sourceId": 1 });
db["$COL"].createIndex( { "rend:sourceId": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "rend:sourceVersionableId": 1 });
db["$COL"].createIndex( { "rend:sourceVersionableId": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "drv:subscriptions.enabled": 1 });
db["$COL"].createIndex( { "drv:subscriptions.enabled": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "collectionMember:collectionIds": 1 });
db["$COL"].createIndex( { "collectionMember:collectionIds": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "nxtag:tags": 1 });
db["$COL"].createIndex( { "nxtag:tags": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "coldstorage:beingRetrieved": 1 });
db["$COL"].createIndex( { "coldstorage:beingRetrieved": 1 }, { sparse: true , background: true} );

// Application specific index
db["$COL"].dropIndex( { "customer:number": 1 });
db["$COL"].createIndex( { "customer:number": 1 }, { sparse: true , background: true} );
db["$COL"].dropIndex( { "account:number": 1 });
db["$COL"].createIndex( { "account:number": 1 }, { sparse: true , background: true} );


""".replace("$COL", col)
js = js + """print("************************"); 
print("All updates completed");"""

createIndex = open("createIndex.js", "w")
n = createIndex.write(js)
createIndex.close()

In [8]:
!mongo "mongodb+srv://nxbench-2826-bench-11-n.vzeqm.mongodb.net/nuxeo" --username $MONGOUSER --password $MONGOPWD createIndex.js

MongoDB shell version v4.4.0
connecting to: mongodb://nxbench-2826-bench-11-n-shard-00-00.vzeqm.mongodb.net:27017,nxbench-2826-bench-11-n-shard-00-01.vzeqm.mongodb.net:27017,nxbench-2826-bench-11-n-shard-00-02.vzeqm.mongodb.net:27017/nuxeo?authSource=admin&compressors=disabled&gssapiServiceName=mongodb&replicaSet=atlas-lwol2l-shard-0&ssl=true
Implicit session: session { "id" : UUID("1e181a9e-cfa4-42ee-94aa-16a1ef651ff8") }
MongoDB server version: 4.2.9
************************
update collection us-east
************************
update collection us-west
************************
update collection archives
************************
All updates completed
